In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [14]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat


planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [2]:
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_ext.tools.http import HttpTool

serper_key = os.getenv("SERPER_API_KEY")

# Validate
if not serper_key:
    raise ValueError("Please set the SERPER_API_KEY environment variable.")

# Use the keys
os.environ['SERPER_API_KEY'] = serper_key

search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."   

In [4]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [5]:
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [6]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    reflect_on_tool_use=False,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [7]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

In [8]:
data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

### Termination Condition

In [9]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination

In [10]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [15]:
planning_agent.description

'An agent for planning tasks, this agent should be the first to engage when given a new task.'

In [16]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [17]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"


In [20]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
Here is the breakdown of the findings based on your query:

1. The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, scoring a total of 1,397 points.
2. Dwyane Wade had 214 total rebounds in the 2007-2008 season.
3. Dwyane Wade had 398 total rebounds in the 2008-2009 season.
4. The percentage change in Dwyane Wade's total rebounds between the 2007-2008 and 2008-2009 seasons is approximately 85.98%.

TERMINATE


TaskResult(messages=[TextMessage(id='e3183700-1e03-4047-a38a-36423c63e69c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 16, 13, 54033, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='96bafa1e-f978-4f47-8e1d-74abef589f37', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=841, completion_tokens=128), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 16, 16, 549309, tzinfo=datetime.timezone.utc), content="Here is the breakdown of the findings based on your query:\n\n1. The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, scoring a total of 1,397 points.\n2. Dwyane Wade had 214 total rebounds in the 2007-2008 season.\n3. Dwyane Wade had 398 total rebounds in the 2008-2009 season.

In [19]:
# With real web search


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
Here is the breakdown of the findings based on your question:

1. The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, scoring a total of 1,397 points.
2. Dwyane Wade had 214 total rebounds in the 2007-2008 season.
3. Dwyane Wade had 398 total rebounds in the 2008-2009 season.
4. The percentage change in Dwyane Wade's total rebounds between the 2007-2008 and 2008-2009 seasons is approximately 85.98%.

TERMINATE


TaskResult(messages=[TextMessage(id='35c16bf9-662b-40c7-ba75-2c8c3660f2da', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 15, 48, 937831, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='85b9322a-e765-4fe8-9ee8-22c4633a3beb', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=657, completion_tokens=128), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 15, 51, 512612, tzinfo=datetime.timezone.utc), content="Here is the breakdown of the findings based on your question:\n\n1. The Miami Heat player with the highest points in the 2006-2007 season was Dwyane Wade, scoring a total of 1,397 points.\n2. Dwyane Wade had 214 total rebounds in the 2007-2008 season.\n3. Dwyane Wade had 398 total rebounds in the 2008-2009 sea

In [21]:
state = await selector_team.save_state()

In [22]:
state


{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'PlanningAgent': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': "To address the request, the task can be broken down into the following subtasks:\n\n1. WebSearchAgent: Determine the player with the highest points for the Miami Heat during the 2006-2007 NBA season.\n2. WebSearchAgent: Locate the total number of rebounds for that player for both the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate the percentage change in total rebounds for that player between the 2007-2008 and 2008-2009 seasons.\n\nLet's proceed with the assignmen

### Custom Selector Function

In [23]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source == web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)

In [25]:
await selector_team.reset()

In [26]:
# With real web search
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your question, we will need to break it down into two parts: 

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Calculate the percentage change in his total rebounds from the 2007-2008 season to the 2008-2009 season.

Let's proceed with these tasks:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Search for the total rebounds of the player from task 1 in the 2007-2008 season and the 2008-2009 season.
3. DataAnalystAgent: Calculate the percentage change in total rebounds between the two seasons found in task 2.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_YxmLU4mg9

TaskResult(messages=[TextMessage(id='9122d752-a0b0-4497-8c8a-fb13f310b804', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 17, 54, 425758, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='cbf2eade-79c0-4ceb-b8cb-b118b4457518', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=160), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 17, 56, 912851, tzinfo=datetime.timezone.utc), content="To address your question, we will need to break it down into two parts: \n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Calculate the percentage change in his total rebounds from the 2007-2008 season to the 2008-2009 season.\n\nLet's proceed with these tasks:\n\n1. We

In [29]:
await selector_team.reset()


In [30]:
# If Data analyst agent is not selected after web search agent
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player's total rebounds during the 2007-2008 and 2008-2009 seasons so that I can calculate the percentage change.
---------- TextMessage (DataAnalystAgent) ----------
To find the Miami Heat player with the highest points in the 2006-2007 season, you may check the records for that season. However, Dwyane Wade was a standout player for the Miami Heat during that time.

Once you provide the data for the total rebounds in the 2007-2008 and 2008-2009 seasons for the player, I can calculate the percentage change in rebounds for you.
---------- TextMessage (DataAnalystAgent) ----------
If you have the data, kindly provide the total rebound statistics for the Miami Heat player during the 2007-2008 and 2008-2009 seasons, and I will calculate the percentage change for you.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data on the total rebounds for the Miami Heat player durin

TaskResult(messages=[TextMessage(id='ff0752eb-27a4-43c6-b5b8-a5cb67a05883', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 19, 20, 100003, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='f5f9a70f-b132-47ae-af85-7afbc3af807e', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=37), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 19, 21, 330460, tzinfo=datetime.timezone.utc), content="Please provide the data for the Miami Heat player's total rebounds during the 2007-2008 and 2008-2009 seasons so that I can calculate the percentage change.", type='TextMessage'), TextMessage(id='63693fd5-3c6f-4566-a28a-c5199a8dad64', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=190, co

In [31]:
await selector_team.reset()

### Candidate Function

In [33]:
from typing import List, Sequence

def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]

In [35]:
# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To answer this question, we need to perform the following tasks:

1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Identify this player's total rebounds for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in this player's total rebounds between the 2007-2008 and 2008-2009 seasons.

Let's start by gathering the necessary information.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_g4f9VOwtEMP2UpeaoE2B7EWR', arguments='{"query":"Miami Heat highest points 2006-2007 season player"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecut

TaskResult(messages=[TextMessage(id='b13c186d-cebc-41c5-a619-87a15d7f26cb', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 40, 30, 774605, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='dd8fc521-82ed-41f4-b319-42a5e1a86c95', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=107), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 40, 32, 317015, tzinfo=datetime.timezone.utc), content="To answer this question, we need to perform the following tasks:\n\n1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent: Identify this player's total rebounds for the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate the percentage ch

In [34]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    

    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    

    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
        

    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
# await sele.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your query, we need to perform the following tasks:

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find total rebounds for this player for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in total rebounds for the identified player between the 2007-2008 and 2008-2009 seasons.

I will start by delegating these tasks.

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.

---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_fG9DOWyvNIuPgauTfMuUQe0s', arguments

TaskResult(messages=[TextMessage(id='6e8c50dc-9448-4a79-a61d-60e6f81d960a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 36, 48, 608056, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c52e8378-2293-4d78-bbbb-f64081c534fe', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=138), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 36, 50, 878076, tzinfo=datetime.timezone.utc), content='To address your query, we need to perform the following tasks:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. WebSearchAgent: Find total rebounds for this player for the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate the percentag